# Power Ranking Systems #

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import nltk
import warnings
warnings.filterwarnings('ignore')

In [33]:
schedule_df = pd.read_csv('nfl_2021.csv')
schedule_df.head()

,Winner/tie,Loser/tie,Pts,Pts.1
0,Tampa Bay Buccaneers,Dallas Cowboys,31,29
1,Philadelphia Eagles,Atlanta Falcons,32,6
2,Pittsburgh Steelers,Buffalo Bills,23,16
3,Carolina Panthers,New York Jets,19,14
4,Cincinnati Bengals,Minnesota Vikings,27,24


In [34]:
schedule_df = schedule_df.rename(columns = {'Pts':'win_pts','Pts.1':'lose_pts'})
schedule_df.head()

,Winner/tie,Loser/tie,win_pts,lose_pts
0,Tampa Bay Buccaneers,Dallas Cowboys,31,29
1,Philadelphia Eagles,Atlanta Falcons,32,6
2,Pittsburgh Steelers,Buffalo Bills,23,16
3,Carolina Panthers,New York Jets,19,14
4,Cincinnati Bengals,Minnesota Vikings,27,24


In [35]:
schedule_df = schedule_df[schedule_df['win_pts'] != schedule_df['lose_pts']]

In [36]:
schedule_df = schedule_df.rename(columns = {'Winner/tie':'Winner','Loser/tie':'Loser'})

In [37]:
team_df = schedule_df['Winner'].value_counts().to_frame().reset_index()


In [38]:
team_df = team_df.rename(columns={'index':'Team','team':'Wins'})

In [39]:
team1 = schedule_df['Loser'].value_counts().to_frame().reset_index()
team1 = team1.rename(columns={'index':'Team','team':'Losses'})
team1.head()

,Team,Loser
0,Jacksonville Jaguars,14
1,Houston Texans,13
2,New York Jets,13
3,New York Giants,13
4,Detroit Lions,13


In [40]:
team_df = team_df.merge(team1,on='Team')
team_df.head(10)

,Team,Winner,Loser
0,Tampa Bay Buccaneers,13,4
1,Green Bay Packers,13,4
2,Dallas Cowboys,12,5
3,Los Angeles Rams,12,5
4,Kansas City Chiefs,12,5
5,Tennessee Titans,12,5
6,Arizona Cardinals,11,6
7,Buffalo Bills,11,6
8,Cincinnati Bengals,10,7
9,New England Patriots,10,7


In [41]:
team_df = team_df.rename(columns = {'Winner':'Wins','Loser':'Losses'})
team_df.columns

Index(['Team', 'Wins', 'Losses'], dtype='object')

In [42]:
team_df['Games']=team_df['Wins'] + team_df['Losses']

In [43]:
team_df['Raw Win%'] = team_df['Wins'] / team_df['Games'] * 100
team_df.head()

,Team,Wins,Losses,Games,Raw Win%
0,Tampa Bay Buccaneers,13,4,17,76.470588
1,Green Bay Packers,13,4,17,76.470588
2,Dallas Cowboys,12,5,17,70.588235
3,Los Angeles Rams,12,5,17,70.588235
4,Kansas City Chiefs,12,5,17,70.588235


In [44]:
schedule_df = schedule_df.merge(team_df,left_on = 'Winner',right_on='Team')
schedule_df.head()

,Winner,Loser,win_pts,lose_pts,Team,Wins,Losses,Games,Raw Win%
0,Tampa Bay Buccaneers,Dallas Cowboys,31,29,Tampa Bay Buccaneers,13,4,17,76.470588
1,Tampa Bay Buccaneers,Atlanta Falcons,48,25,Tampa Bay Buccaneers,13,4,17,76.470588
2,Tampa Bay Buccaneers,New England Patriots,19,17,Tampa Bay Buccaneers,13,4,17,76.470588
3,Tampa Bay Buccaneers,Miami Dolphins,45,17,Tampa Bay Buccaneers,13,4,17,76.470588
4,Tampa Bay Buccaneers,Philadelphia Eagles,28,22,Tampa Bay Buccaneers,13,4,17,76.470588


In [45]:
schedule_df = schedule_df.merge(team_df,left_on = 'Loser',right_on='Team')
schedule_df.head()

,Winner,Loser,win_pts,lose_pts,Team_x,Wins_x,Losses_x,Games_x,Raw Win%_x,Team_y,Wins_y,Losses_y,Games_y,Raw Win%_y
0,Tampa Bay Buccaneers,Dallas Cowboys,31,29,Tampa Bay Buccaneers,13,4,17,76.470588,Dallas Cowboys,12,5,17,70.588235
1,Arizona Cardinals,Dallas Cowboys,25,22,Arizona Cardinals,11,6,17,64.705882,Dallas Cowboys,12,5,17,70.588235
2,Kansas City Chiefs,Dallas Cowboys,19,9,Kansas City Chiefs,12,5,17,70.588235,Dallas Cowboys,12,5,17,70.588235
3,Denver Broncos,Dallas Cowboys,30,16,Denver Broncos,7,10,17,41.176471,Dallas Cowboys,12,5,17,70.588235
4,Las Vegas Raiders,Dallas Cowboys,36,33,Las Vegas Raiders,10,7,17,58.823529,Dallas Cowboys,12,5,17,70.588235


In [46]:
schedule_df = schedule_df.rename(columns = {'Raw Win%_x':'Raw Win%_winner','Raw Win%_y':'Raw Win%_loser'})
schedule_df.head()

,Winner,Loser,win_pts,lose_pts,Team_x,Wins_x,Losses_x,Games_x,Raw Win%_winner,Team_y,Wins_y,Losses_y,Games_y,Raw Win%_loser
0,Tampa Bay Buccaneers,Dallas Cowboys,31,29,Tampa Bay Buccaneers,13,4,17,76.470588,Dallas Cowboys,12,5,17,70.588235
1,Arizona Cardinals,Dallas Cowboys,25,22,Arizona Cardinals,11,6,17,64.705882,Dallas Cowboys,12,5,17,70.588235
2,Kansas City Chiefs,Dallas Cowboys,19,9,Kansas City Chiefs,12,5,17,70.588235,Dallas Cowboys,12,5,17,70.588235
3,Denver Broncos,Dallas Cowboys,30,16,Denver Broncos,7,10,17,41.176471,Dallas Cowboys,12,5,17,70.588235
4,Las Vegas Raiders,Dallas Cowboys,36,33,Las Vegas Raiders,10,7,17,58.823529,Dallas Cowboys,12,5,17,70.588235


In [47]:
schedule_df['raw win diff'] = schedule_df['Raw Win%_winner'] - schedule_df['Raw Win%_loser']

In [68]:
schedule_df['adj win'] = schedule_df['Raw Win%_winner'] - 50
schedule_df['adj lose'] = schedule_df['Raw Win%_loser'] - 50

In [69]:
schedule_df.head()

,Winner,Loser,win_pts,lose_pts,Team_x,Wins_x,Losses_x,Games_x,Raw Win%_winner,Team_y,Wins_y,Losses_y,Games_y,Raw Win%_loser,raw win diff,adj win,adj lose
0,Tampa Bay Buccaneers,Dallas Cowboys,31,29,Tampa Bay Buccaneers,13,4,17,76.470588,Dallas Cowboys,12,5,17,70.588235,5.882353,26.470588,20.588235
1,Arizona Cardinals,Dallas Cowboys,25,22,Arizona Cardinals,11,6,17,64.705882,Dallas Cowboys,12,5,17,70.588235,-5.882353,14.705882,20.588235
2,Kansas City Chiefs,Dallas Cowboys,19,9,Kansas City Chiefs,12,5,17,70.588235,Dallas Cowboys,12,5,17,70.588235,0.000000,20.588235,20.588235
3,Denver Broncos,Dallas Cowboys,30,16,Denver Broncos,7,10,17,41.176471,Dallas Cowboys,12,5,17,70.588235,-29.411765,-8.823529,20.588235
4,Las Vegas Raiders,Dallas Cowboys,36,33,Las Vegas Raiders,10,7,17,58.823529,Dallas Cowboys,12,5,17,70.588235,-11.764706,8.823529,20.588235


In [72]:
t1 = schedule_df[['Winner','adj lose']]
t1 = t1.rename(columns = {'Winner': 'Team','adj lose':'opponent adj'})
t2 = schedule_df[['Loser','adj win']]
t2 = t2.rename(columns = {'Loser': 'Team', 'adj win' : 'opponent adj'})
t3 = pd.concat([t1,t2])
t3.head(20)

,Team,opponent adj
0,Tampa Bay Buccaneers,20.588235
1,Arizona Cardinals,20.588235
2,Kansas City Chiefs,20.588235
3,Denver Broncos,20.588235
4,Las Vegas Raiders,20.588235
5,Tampa Bay Buccaneers,-8.823529
6,Tampa Bay Buccaneers,-8.823529
7,Philadelphia Eagles,-8.823529
8,Carolina Panthers,-8.823529
9,San Francisco 49ers,-8.823529


In [77]:
t3_mean = t3.groupby('Team')['opponent adj'].mean().to_frame().reset_index()

In [78]:
t3_mean.head(20)

,Team,opponent adj
0,Arizona Cardinals,-1.146194
1,Atlanta Falcons,-2.876298
2,Baltimore Ravens,3.049308
3,Buffalo Bills,-2.746540
4,Carolina Panthers,0.865052
5,Chicago Bears,2.227509
6,Cincinnati Bengals,-2.833045
7,Cleveland Browns,1.319204
8,Dallas Cowboys,-1.211073
9,Denver Broncos,-1.643599


In [79]:
team_df = team_df.merge(t3_mean,on='Team')
team_df.head()

,Team,Wins,Losses,Games,Raw Win%,opponent adj
0,Tampa Bay Buccaneers,13,4,17,76.470588,-3.287197
1,Green Bay Packers,13,4,17,76.470588,-2.270761
2,Dallas Cowboys,12,5,17,70.588235,-1.211073
3,Los Angeles Rams,12,5,17,70.588235,-1.838235
4,Kansas City Chiefs,12,5,17,70.588235,3.827855


In [80]:
team_df = team_df.rename(columns = {'opponent adj':'round 1 adj'})
team_df['round 1 win %'] = team_df['Raw Win%'] + team_df['round 1 adj']
team_df.head()

,Team,Wins,Losses,Games,Raw Win%,round 1 adj,round 1 win %
0,Tampa Bay Buccaneers,13,4,17,76.470588,-3.287197,73.183391
1,Green Bay Packers,13,4,17,76.470588,-2.270761,74.199827
2,Dallas Cowboys,12,5,17,70.588235,-1.211073,69.377163
3,Los Angeles Rams,12,5,17,70.588235,-1.838235,68.750000
4,Kansas City Chiefs,12,5,17,70.588235,3.827855,74.416090


In [82]:
schedule_df = schedule_df.merge(team_df[['Team','round 1 win %']],left_on = 'Winner',right_on='Team')
schedule_df = schedule_df.merge(team_df[['Team','round 1 win %']],left_on = 'Loser',right_on='Team')
schedule_df.head()

,Winner,Loser,win_pts,lose_pts,Team_x,Wins_x,Losses_x,Games_x,Raw Win%_winner,Team_y,...,Losses_y,Games_y,Raw Win%_loser,raw win diff,adj win,adj lose,Team_x,round 1 win %_x,Team_y,round 1 win %_y
0,Tampa Bay Buccaneers,Dallas Cowboys,31,29,Tampa Bay Buccaneers,13,4,17,76.470588,Dallas Cowboys,...,5,17,70.588235,5.882353,26.470588,20.588235,Tampa Bay Buccaneers,73.183391,Dallas Cowboys,69.377163
1,Arizona Cardinals,Dallas Cowboys,25,22,Arizona Cardinals,11,6,17,64.705882,Dallas Cowboys,...,5,17,70.588235,-5.882353,14.705882,20.588235,Arizona Cardinals,63.559689,Dallas Cowboys,69.377163
2,Kansas City Chiefs,Dallas Cowboys,19,9,Kansas City Chiefs,12,5,17,70.588235,Dallas Cowboys,...,5,17,70.588235,0.000000,20.588235,20.588235,Kansas City Chiefs,74.416090,Dallas Cowboys,69.377163
3,Denver Broncos,Dallas Cowboys,30,16,Denver Broncos,7,10,17,41.176471,Dallas Cowboys,...,5,17,70.588235,-29.411765,-8.823529,20.588235,Denver Broncos,39.532872,Dallas Cowboys,69.377163
4,Las Vegas Raiders,Dallas Cowboys,36,33,Las Vegas Raiders,10,7,17,58.823529,Dallas Cowboys,...,5,17,70.588235,-11.764706,8.823529,20.588235,Las Vegas Raiders,59.883218,Dallas Cowboys,69.377163


In [83]:
schedule_df['adj2 win'] = schedule_df['round 1 win %_x'] - 50
schedule_df['adj2 lose'] = schedule_df['round 1 win %_y'] - 50

t1 = schedule_df[['Winner','adj2 lose']]
t1 = t1.rename(columns = {'Winner': 'Team','adj2 lose':'opponent adj2'})
t2 = schedule_df[['Loser','adj2 win']]
t2 = t2.rename(columns = {'Loser': 'Team', 'adj2 win' : 'opponent adj2'})
t3 = pd.concat([t1,t2])
t3.head(20)

,Team,opponent adj2
0,Tampa Bay Buccaneers,19.377163
1,Arizona Cardinals,19.377163
2,Kansas City Chiefs,19.377163
3,Denver Broncos,19.377163
4,Las Vegas Raiders,19.377163
5,Tampa Bay Buccaneers,-11.699827
6,Tampa Bay Buccaneers,-11.699827
7,Philadelphia Eagles,-11.699827
8,Carolina Panthers,-11.699827
9,San Francisco 49ers,-11.699827


In [84]:
t3_mean = t3.groupby('Team')['opponent adj2'].mean().to_frame().reset_index()
team_df = team_df.merge(t3_mean,on='Team')

team_df = team_df.rename(columns = {'opponent adj2':'round 2 adj'})
team_df['round 2 win %'] = team_df['round 1 win %'] + team_df['round 2 adj']
team_df.head()

,Team,Wins,Losses,Games,Raw Win%,round 1 adj,round 1 win %,round 2 adj,round 2 win %
0,Tampa Bay Buccaneers,13,4,17,76.470588,-3.287197,73.183391,-3.679015,69.504376
1,Green Bay Packers,13,4,17,76.470588,-2.270761,74.199827,-0.924842,73.274985
2,Dallas Cowboys,12,5,17,70.588235,-1.211073,69.377163,-0.954101,68.423061
3,Los Angeles Rams,12,5,17,70.588235,-1.838235,68.750000,-1.525290,67.224710
4,Kansas City Chiefs,12,5,17,70.588235,3.827855,74.416090,3.881284,78.297374


In [93]:
team_df['SOS'] = team_df['round 2 adj'].rank(ascending = False)
team_df.sort_values('SOS').head()

,Team,Wins,Losses,Games,Raw Win%,round 1 adj,round 1 win %,round 2 adj,round 2 win %,SOS
13,Pittsburgh Steelers,9,7,16,56.250000,4.044118,60.294118,4.303633,64.597751,1.0
4,Kansas City Chiefs,12,5,17,70.588235,3.827855,74.416090,3.881284,78.297374,2.0
18,Baltimore Ravens,8,9,17,47.058824,3.049308,50.108131,3.425860,53.533991,3.0
28,New York Giants,4,13,17,23.529412,3.633218,27.162630,3.279564,30.442194,4.0
22,Washington Football Team,7,10,17,41.176471,2.941176,44.117647,2.671484,46.789131,5.0


In [104]:
def simulate_EV(df,team1,team2):
    t1 = df[df['Team'] == team1]['round 2 win %'].item()
    t2 = df[df['Team'] == team2]['round 2 win %'].item()
    t1_delta = t1-50
    t2_delta = t2-50
    
    t1_delta2 = t1_delta - t2_delta
    t2_delta2 = t2_delta - t1_delta
    
    ev1 = t1_delta2 + 50
    ev2 = t2_delta2 + 50
    
    print(f"{team1} vs {team2}: ")
    print(f"Odds {team1} wins: {round(ev1,2)}%")
    print(f"Odds {team2} wins: {round(ev2,2)}%")
    

In [103]:
simulate_EV(team_df,'Pittsburgh Steelers','Kansas City Chiefs')

Pittsburgh Steelers vs Kansas City Chiefs: 
Odds Pittsburgh Steelers wins: 36.3 %
Odds Kansas City Chiefs wins: 63.7 %


In [106]:
test_df = pd.read_csv('test_data.csv')
test_df.head()

,Winner,Loser,Pts_win,Pts_lose
0,Cincinnati Bengals,Las Vegas Raiders,26,19
1,Buffalo Bills,New England Patriots,47,17
2,Tampa Bay Buccaneers,Philadelphia Eagles,31,15
3,San Francisco 49ers,Dallas Cowboys,23,17
4,Kansas City Chiefs,Pittsburgh Steelers,42,21


In [109]:
def test_ev_prediction(df,test_data):
    winners = test_data['Winner'].to_list()
    losers = test_data['Loser'].to_list()
    predictions = []
    for i in range(0,len(winners)):
        t1 = df[df['Team'] == winners[i]]['round 2 win %'].item()
        t2 = df[df['Team'] == losers[i]]['round 2 win %'].item()
        t1_delta = t1-50
        t2_delta = t2-50
        t1_delta2 = t1_delta - t2_delta
        t2_delta2 = t2_delta - t1_delta
        ev1 = t1_delta2 + 50
        ev2 = t2_delta2 + 50
        if ev1 > ev2:
            predictions.append(winners[i])
        elif ev2 > ev1:
            predictions.append(losers[i])
    test_results = test_data
    test_results['prediction'] = predictions
    test_results['match'] = np.where(test_results['prediction'] == test_results['Winner'],1,0)
    return test_results    
     

In [110]:
results = test_ev_prediction(team_df,test_df)

In [111]:
results

,Winner,Loser,Pts_win,Pts_lose,prediction,match
0,Cincinnati Bengals,Las Vegas Raiders,26,19,Las Vegas Raiders,0
1,Buffalo Bills,New England Patriots,47,17,Buffalo Bills,1
2,Tampa Bay Buccaneers,Philadelphia Eagles,31,15,Tampa Bay Buccaneers,1
3,San Francisco 49ers,Dallas Cowboys,23,17,Dallas Cowboys,0
4,Kansas City Chiefs,Pittsburgh Steelers,42,21,Kansas City Chiefs,1
5,Los Angeles Rams,Arizona Cardinals,34,11,Los Angeles Rams,1
6,Cincinnati Bengals,Tennessee Titans,19,16,Tennessee Titans,0
7,San Francisco 49ers,Green Bay Packers,13,10,Green Bay Packers,0
8,Los Angeles Rams,Tampa Bay Buccaneers,30,27,Tampa Bay Buccaneers,0
9,Kansas City Chiefs,Buffalo Bills,42,36,Kansas City Chiefs,1


In [113]:
print(f"ELO Model prediction success in playoffs: {round(results['match'].mean() * 100,2)}%")

ELO Model prediction success in playoffs: 53.85%
